In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math
from student_utils import *
from utils import *
import community
from queue import PriorityQueue
# from ipynb.fs.full.gen_inputs import add_weights2

# Building outputs from all algorithms

## Select the best output for each input

In [ ]:
def select_outputs(pickle_dir):
    pfiles = get_files_with_extension(pickle_dir, 'p')
    dic = {}
    meta_dic = {}
    for pf in pfiles:
        pobj = load_obj(pf)
        for item in pobj:
            # key=<./path/to/input_file>.in, value=(<./path/to/output_file.out>, cost)
            input_file = item[0]
            output_file = item[1]
            cost = item[2][0]
            if input_file in dic:
                dic[input_file].append((output_file, cost))
            else:
                dic[input_file] = [(output_file, cost)]
    if not os.path.isdir('final/outputs'):
        os.mkdir('final')
        os.mkdir('final/outputs')
    for lst in dic.values():
        best_output, best_cost = min(lst, key=lambda x: x[1])
        os.system('cp ' + best_output + ' final/outputs/')
        meta_dic[best_output] = best_cost
    save_obj(meta_dic, 'final/meta.p')
    write_data_to_file('final/meta.txt', meta_dic.items(), '\n')

In [ ]:
select_outputs('pickle_output')

# Metric TSP Solver

## Method 1: A polynomial-time Algo with cost <= 2*optimal

1. Take the MST of G.
2. Do a DPS on MST to get PT(pseudotour).
3. Go from PT to T* (Take each vertex only the first time it appears in PT)

In [ ]:
G1 = nx.complete_graph(10)
add_weights2(G1, 1, 10)
print(list(G1.edges().data()))
plt.subplot()
nx.draw(G1, node_color='yellow', with_labels=True)

In [ ]:
ta_homes = [0,1,2,3,4]
G1_sub = G1.subgraph(ta_homes)
print(list(G1_sub.edges().data()))
plt.subplot()
nx.draw(G1_sub, node_color='yellow', with_labels=True)

In [ ]:
PT = nx.minimum_spanning_tree(G1_sub)
# print(sorted(PT.edges(data=True)))
print(PT.edges(data=True))

plt.subplot()
nx.draw(PT, node_color='yellow', with_labels=True)

In [ ]:
# TODO: implement dfs yielding pseudotour
list(nx.dfs_edges(PT, source=0))

# Rao drives every TA home: this is a TSP

## Nearest Neighbor Algorithm

In [ ]:
def nn_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph1, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    current_idx = start_idx
    while homes_idx:
        lst_dist = [(h_idx, nx.shortest_path_length(graph1, current_idx, h_idx, 'weight')) for h_idx in homes_idx]
        idx, _ = min(lst_dist, key=lambda x: x[1])
        shortest_path = nx.shortest_path(graph1, current_idx, idx, 'weight')
        shortest_path.pop()
        path.extend(shortest_path)
        drop_offs[idx] = [idx]
        current_idx = idx
        homes_idx.remove(idx)
    last_shortest_path = nx.shortest_path(graph1, current_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs

In [ ]:
data = read_file('test_inputs/27_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
nn_solve(lst_loc, lst_house, start_loc, adj_mat)

## 2-Approximation using MST

- If the MST is connected then we can perform DFS and construct the path.
- If the MST is not connected then we need to connect all homes by finding shortest paths between them.
- To get the final path we look at each path in the list and check in the original graph, if there is an edge then we do nothing, but if it does not have an edge then we replace it with the shortest path between those nodes.

In [ ]:
def connect_homes(graph, nodes):
    newgraph = nx.Graph()
    newgraph.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            w = nx.shortest_path_length(graph, nodes[i], nodes[j], 'weight')
            newgraph.add_edge(nodes[i], nodes[j], weight=w)
    return newgraph
    
def mst_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    nodes = homes_idx + [start_idx]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    newgraph = graph.subgraph(nodes)
    if not nx.is_connected(newgraph):
        newgraph = connect_homes(graph, nodes)
    mst = nx.minimum_spanning_tree(newgraph, 'weight')
    dfs = list(nx.dfs_edges(mst, start_idx))
    path = [start_idx]
    drop_offs = {}
    if start_idx in homes_idx:
        drop_offs[start_idx] = [start_idx]
    current_idx = -1
    while dfs:
        edge = dfs.pop(0)
        current_idx = edge[1]
        if current_idx not in path:
            path.append(current_idx)
            drop_offs[current_idx] = [current_idx]
    final_path = [path[0]]
    for i in range(len(path)-1):
        if graph.has_edge(path[i], path[i+1]):
            final_path.append(path[i+1])
        else:
            final_path.pop()
            final_path.extend(nx.shortest_path(graph, path[i], path[i+1], 'weight'))
    last_shortest_path = nx.shortest_path(graph, final_path[-1], start_idx, 'weight')
    final_path.pop()
    final_path.extend(last_shortest_path)
    return final_path, drop_offs

In [ ]:
# data = read_file('test_inputs/1_50.in')
data = read_file('inputs/79_200.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
mst_solve(lst_loc, lst_house, start_loc, adj_mat)

## 1.5-Approximation using Christofides Algorithm

In [ ]:
#code

In [ ]:
#test

## Branch-and-bound using the algorithm in the textbook

In [ ]:
#code

In [ ]:
#test

## Branch-and-bound using another lower-bound

In [ ]:
#code

In [ ]:
#test

# Rao drives TAs to some dropoff locations (clustering)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
start_idx = list_of_locations.index(starting_car_location)
homes_idx = [list_of_locations.index(h) for h in list_of_homes]

plt.figure(1)
nx.draw(graph, with_labels=True)

#first compute the best partition
partition = community.best_partition(graph)

#drawing
plt.figure(2)
size = float(len(set(partition.values())))
pos = nx.spring_layout(graph)
count = 0.
for com in set(partition.values()) :
#     print(type(com))
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(graph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))
print('Number of communities: ', count)

nx.draw_networkx_edges(graph, pos, alpha=0.5)
plt.show()

In [ ]:
def count_homes(k, graph, homes_idx):
    lst = []
    for node in graph.nodes():
        lst_homes = []
        neighbors = list(graph.neighbors(node))
        for n in neighbors:
            if n in homes_idx:
                lst_homes.append(n)
        if k == 2:
            for n in neighbors:
                for nn in graph.neighbors(n):
                    if nn in homes_idx:
                        lst_homes.append(nn)
        if len(lst_homes) > 0:
            lst.append((node, lst_homes))
    return lst

def init_pq(lst, graph, cur_idx):
    shortest_paths = nx.shortest_path_length(graph, cur_idx, weight='weight')
    pq = PriorityQueue()
    for i in lst:
        node = i[0]
        pq.put((shortest_paths[node], i))
    return pq
    
def k_layers_cluster(k, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    lst = count_homes(k, graph, homes_idx)
    cur_idx = start_idx
    while homes_idx:
        pq = init_pq(lst, graph, cur_idx)
        cost, center_homes = pq.get()
        center = center_homes[0]
        lst_homes = center_homes[1]
        lst_copy = lst_homes.copy()
        for h in lst_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                lst_homes.remove(h)
        if lst_homes:
            shortest_path = nx.shortest_path(graph, cur_idx, center, 'weight')
            shortest_path.pop()
            path.extend(shortest_path)
            drop_offs[center] = lst_homes
            cur_idx = center
        lst.remove(center_homes)
    last_shortest_path = nx.shortest_path(graph, cur_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs
    
def nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    return k_layers_cluster(1, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)